In [1]:
import tempfile
import subprocess
import shlex
import getopt
import sys
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
#from Bio.SeqUtils import GC
import time# import time, gmtime, strftime
import os
import shutil
import pandas as pd
import csv
#from datetime import datetime
import numpy as np
import re
from itertools import islice       

In [136]:
def batch_iterator(iterator, batch_size):
    """Returns lists of length batch_size.

    This can be used on any iterator, for example to batch up
    SeqRecord objects from Bio.SeqIO.parse(...), or to batch
    Alignment objects from Bio.AlignIO.parse(...), or simply
    lines from a file handle.

    This is a generator function, and it returns lists of the
    entries from the supplied iterator.  Each list will have
    batch_size entries, although the final list may be shorter.
    """
    entry = True  # Make sure we loop once
    while entry:
        batch = []
        while len(batch) < batch_size:
            try:
                entry = next(iterator)
            except StopIteration:
                entry = None
            if entry is None:
                # End of file
                break
            batch.append(entry)
        if batch:
            yield batch
            
def batch_iterator3(iterator, batch_size):
    """Returns lists of length batch_size.

       a modified version of batch_iterator() which replaces original fasta id 
       with the positional fasta id, and stores in a temporary file
       count  starts with 1
    """
    entry = True  # Make sure we loop once
    counter=0
    while entry:
        batch_names=[]
        batch = []
        while len(batch) < batch_size:
            try:
                #origianlly was iterator.next()
                #changed for py3 compatibility
                #needs python 2.6+
                entry = next(iterator)
            except StopIteration:
                entry = None
            if entry is None:
                # End of file
                break
            counter+=1
            #entry._id=entry.id
            entry.id=str(counter)
            batch.append(entry)
            
        if batch:
            yield batch
            
def joinDescriptionColumns(descr_columns):
    merged=''
    for row in descr_columns.dropna():
        if (row != None):
            #print(merged)
            merged+=row+' '
    #print(descr_columns)
    return merged.strip()

def usage():
    print("\nThis is the usage function\n")
#    print 'Usage: '+sys.argv[0]+' -i <input_file> [-o <output>] [-l <minimum length>]'
#    print 'Example: '+sys.argv[0]+' -i input.fasta -o output.fasta -l 100'


    
def extractFeatures(ids, filename,md):
    features=[]
    pos=0
    for record in SeqIO.parse(filename,"fasta"):
        pos+=1
        if record.name in ids:
            features.append({'qid':record.id, 'position':pos, 'descr':record.description, 'seq':str(record.seq)})
    return pd.DataFrame(features)

def extractFeatures2(ids, filename):#putting in a seq object instead of string
    features=[]
    pos=0
    for record in SeqIO.parse(filename,"fasta"):
        pos+=1
        if record.name in ids:
            features.append({'qid':record.id, 'position':pos, 'descr':record.description, 'seq':record})
    return pd.DataFrame(features)

def is_operon(x):
    return x['diff1'] or x['diff2']

def rel_coordinates(x, md):
    
    
    if ((x['diff1'] < md) or (x['diff2']) <md):
        return min(x['diff1'],x['diff2'])
    else:
        return 0
        
    #extracting features by positions with no looping through the whole file
def extractFeatures4(positions, generator):#putting in a seq object instead of string
    #file_iter = SeqIO.parse(open(filename),"fasta")#m.rosea
    offset=0
    features=[]
    #entries=list()
    for i in range(len(positions)):
        position=positions[i]-offset
        offset=positions[i]
        record=next(islice(generator, position-1,position))#0 start of a generator coupled with 1 start of fasta ids
        
        features.append({'qid':record.id, 'position':positions[i], 'descr':record.description, 'seq':record})
    return pd.DataFrame(features)

#this function combines adjacent operons(fragments) into one    input_file = '../76969.assembled.faa'

def operonCount(lst):
    opCnt=0
    state=False
    for i in xrange(len(lst)):
        if lst[i]:
            newState=True
            if (state==False) and (newState==True):
                opCnt+=1
            lst[i]=opCnt
            state=newState
        else:
            state=False
    return lst

#this function splits adjacent operons, based on distance
#for the operons of large size it will artificially break it into pieces

def operonCount2(lst, pos, md, min_operon_size):
    opCnt=0
    state=False
    position=0
    
    
    for i in range(len(lst)):
        if lst[i]:
            newState=True
            #position=pos[i]
            if (state==False) and (newState==True):
                position=pos[i]
                opCnt+=1
            if (pos[i]>position+md+min_operon_size+5):#!!! needs a fix so large operons won't be affected
                opCnt+=1
    
            lst[i]=opCnt
            state=newState
        else:
            state=False
    return lst    
    
    
    
    
    
def analyzeStructure(name_string, filter_operons={'CAB':0,'ABC':0}):#fix this for the extended version
    for key in filter_operons.keys():
    
        if key in name_string:
            filter_operons[key]=1
            return (key, 1)
        if key[::-1] in name_string:
            filter_operons[key]=-1
            return (key, -1)
    return (None, None)

def concatSeparatedValues(row, sep=" "):
    concatVal=''
    for i in range(output['type'].iloc[1].size):
        concatVal+=str(row.iloc[i])+sep
    concatVal=concatVal[:-len(sep)]#removes last unnesessary separator
    return concatVal.rstrip()

def operonParser(op_lst, input_file, op_type=False, full_operons=False, remove_terminal_stop_codon=True):
    
    
    output_lst=list()#list of seq elements for fasta exporting
    out=pd.DataFrame()
    
    #initializing output variables
    if op_lst == []:
        return 1
    if full_operons:
        out= pd.DataFrame(columns=['operon #','input_file','type',  'rel_distance','size', 'concat_ids','concat_eval', 'concat_descr', 'C', 'A', 'B', 'full_operon'])
    out= pd.DataFrame(columns=['operon #','input_file','type','rel_distance','concat_eval','concat_ids','size',  'concat_descr', 'C', 'A', 'B'])
    
    for operon in op_lst:
        
        #print(operon)
        operon_structure=''.join(operon['type'].tolist())
        #print(operon_structure)
        op_type,forward=analyzeStructure(operon_structure)
        if op_type:
            #converting evalue to float
            operon[4]=operon[4].astype(float)#for correct sorting
            #reverse operon if needed
            #print(operon)
            if forward == -1:
                #''.join(operons[1].reset_index(drop=True)['type'].sort_index(ascending=False).tolist())
                operon=operon.reset_index(drop=True).sort_index(ascending=False)#rookie way   
            else:
                operon=operon.reset_index(drop=True) #else clause is a bit unnecessary
            #print(type(operon))
            cnt=int(operon['operon_count'].mean())#!!! needs an assert for integer value
            inp_file=input_file
            
            #ot=operon_structure #inclues extended understanding of an operon
            ot=op_type
            
            
            rd=''.join(operon.loc[:, 'rel_coordinates'].astype(str).tolist())
            sz=len(operon.index)
            
            
            idx_a=operon[operon.loc[:,'type']=='A'].sort_values(4, ascending=True).head(1).index[0]
            idx_b=operon[operon.loc[:,'type']=='B'].sort_values(4, ascending=True).head(1).index[0]
            idx_c=operon[operon.loc[:,'type']=='C'].sort_values(4, ascending=True).head(1).index[0]
            
            
            
            concat_eval= ";".join([str(x) for x in operon.loc[[idx_c, idx_a, idx_b]][4].astype(str)])
            #peron
            
            
            concat_ids=";".join([x.id for x in operon.loc[[idx_c, idx_a, idx_b]]['seq']])
            concat_descr=";".join([x.description for x in operon.loc[[idx_c, idx_a, idx_b]]['seq']])
            #str(operons[1][operons[1]['type']=='C']['seq'].values[0].seq)
            #takes one value from operon with the smallest e-vaue if multiple values are present
            #str(record.seq)
#             c_seq=str(operon[operon.loc[:,'type']=='C'].sort_values(4, ascending=True).head(1)['seq'].values[0].seq)
#             a_seq=str(operon[operon.loc[:,'type']=='A'].sort_values(4, ascending=True).head(1)['seq'].values[0].seq)
#             b_seq=str(operon[operon.loc[:,'type']=='B'].sort_values(4, ascending=True).head(1)['seq'].values[0].seq)


            #removing terminal stop codon symbol for concatenation
            c_seq=str(operon.loc[idx_c]['seq'].seq)
            a_seq=str(operon.loc[idx_a]['seq'].seq)
            b_seq=str(operon.loc[idx_b]['seq'].seq)
            
            if remove_terminal_stop_codon:
                if c_seq[-1] == "*":
                    c_seq=c_seq[:-1]
                if a_seq[-1] == "*":
                    a_seq=a_seq[:-1]
                if b_seq[-1] == "*":
                    b_seq=b_seq[:-1]
        
            record=SeqRecord(Seq(c_seq+a_seq+b_seq, IUPAC.protein), id=concat_ids, description=concat_descr)
#           records.append(record)
            
            output_lst.append(record)
#             if full_operons:
#                 full_op=
            
            
            out=out.append(pd.Series({'operon #':cnt, 
                                  'input_file':inp_file, 
                                  'type':ot, 
                                  'rel_distance':rd, 
                                  'size': sz,
                                  'concat_eval': concat_eval,
                                  'concat_ids': concat_ids,
                                  'concat_descr': concat_descr,
                                  'C':c_seq,
                                  'A':a_seq,
                                  'B':b_seq}), ignore_index=True)
                                  
          
            
    return out, output_lst     

# all in one function, identifies and counts
# all in one function, identifies and counts
def operonCount5(inp, position=1):#recursive operon count
    '''
    takes a series, np.array or list of sorted positions as an input and returns 
    a numpy array of integers, where 0 indicates  no operon, and a positive value indicates operon number
    
    '''
    beginning=True
    inp_size=len(inp)
    out=np.empty([inp_size])
    out[0]=0 # taking care of the initial value
    op_cnt=0
    operon=0
    while position < (inp_size): # i < 203## we have 1 - 202, and checking for previous position
        

        if (inp[position] - inp[position-1]) <= min_operon_size:
            
            if (operon==0): #if starting a new operon appending a previous position
                op_cnt+=1
                out[position-1]=op_cnt #capturing initial position, and re-writing previous False
            operon=op_cnt
        else:
            operon=0
        out[position]=operon
        position+=1
    return out

# Body of the main function: Part 1

In [166]:
model='models/amoCAB'
max_distance=3
min_operon_size=2# parameter needs more thorough work
chunk_size=30000
#operon_type=['cab','abc']# use "!" symbol to exclude operons
operon_type=['all']# use "!" symbol to exclude operons
output_name=''
e_val = 1e-3

#initialize to work as a prototype
input_file='bugs/43209.assembled.faa'
#global model, max_distance, min_operon_size
# try:                                
#     opts, args = getopt.getopt(argv, "i:o:e:s:t:m:h", ["input=","output=","e_value=","size=","type=","model=","fragment_size=", "help"])
# except getopt.GetoptError:          
#     usage()                         
#     sys.exit(2)                     
# for opt, arg in opts:                
#     if opt in ("-h", "--help"):      
#         usage()              
#         sys.exit() 
# #        elif opt in ("--recover_after_failure"):
# #            recover_after_failure = True
# #            print "Recover after failure:", recover_after_failure
#     elif opt in ("-o", "--output"):
#         if arg:
#             output_name=arg.strip()
#     elif opt in ("-e", "--e_value"):
#         try:
#             e_val = float(arg)
#         except:
#             print "\nERROR: Please enter numerical value as -e parameter (default: 1e-3)"
#             usage()
#             sys.exit(1)
#     elif opt in ("-s", "--size"):
#         if arg:
#             try:
#                 min_operon_size=int(arg.strip())
#             except:
#                 print("ERROR: Please pass integer value to operon [--size] parameter")
#                 sys.exit(1)

#     elif opt in ('-t', '--type'):
#         if arg:
#             operon_type=arg.strip().split(',')
#             #print('Operon type:',operon_type) #debugging message

#     elif opt in ("--fragment_size"):
#         if arg:
#             try:
#                 chunk_size=abs(int(arg.strip()))
#             except:
#                 print("ERROR: Please pass integer value to operon [--fragmen_size] parameter")
#                 sys.exit(1)




#     elif opt in ("-i", "--input"):
#         if arg:
#             input_file=arg.strip()
#             #infiles = arg


#            
#    
try:
    #
    with open(input_file, "rU") as hand_ref:
        pass
except:
    print("\nERROR: Input File ["+input_file+"] doesn't exist")
    usage()
    sys.exit(1)




files=list()
sizes={}

try:
    record_iter = SeqIO.parse(open(input_file),"fasta")
except:
    print("\nERROR: Could not parse Input File ["+input_file+"]")

basename=os.path.basename(input_file)



#    for i, batch in enumerate(batch_iterator(record_iter, 30000)): #keeps original ids
for i, batch in enumerate(batch_iterator3(record_iter, chunk_size)): #uses positional ids

    label = i
    #f = tempfile.NamedTemporaryFile(delete=False)#exists on closing
    f = tempfile.NamedTemporaryFile(mode='w+t')#deleted after f.close()
    files.append((label,f))
    #f.seek(0)
    count = SeqIO.write(batch, f, "fasta")
    f.flush() #this solves the EOF problem
#     with open(filename, "w") as handle:
#         count = SeqIO.write(batch, handle, "fasta")
    #print("Wrote %i records to %s" % (count, f.name))
    sizes[f.name]=count






#creating dictionary containing labels # may be entirely useles/redundant
files2={j.name:i for i,j in files}


#main heavylifting part

#hmmscan --tblout amocab_hmmscan_mros.tab  amoCAB 2517287028.genes.faa > /dev/null
#os.system("makeblastdb -in "+input_ref_0+" -dbtype nucl -title "+title_db+" -out "+outfile_db+" -parse_seqids")
fnames=""
flabels=""
for (l,f) in files:
    fnames+=f.name+" "
    flabels+=str(l)+" "
#print flabels
    #print(l)
    #print(f.name)
    #parallel -j $1 blastall "-p blastn -d nt/nt -i "{}" -o "{.}".xml -e 1e-10 -m 7 -K 100 -b 200" ::: *.fa
    #print('Exists after close:', os.path.exists(f.name))
#this prints nicely to stdout, and is caught by p.stdout.read()
#cmd="parallel -j 8 hmmscan "+"-o /dev/null --noali --cpu 1"+" --tblout >(tee /dev/stdout)  " + "../amoCAB {} ::: "+fnames
#this prints to the file and stdout
#0.tab should be deleted if exists
if os.path.exists("0.tab"):
    os.remove("0.tab")
cmd="parallel -j 8 hmmscan "+"-o /dev/null --noali --cpu 1 -E "+str(e_val)+" --tblout >(tee -a 0.tab)  " + model+" {} ::: "+fnames
args = shlex.split(cmd)
#os.system(cmd)
p=subprocess.Popen(args,stdout=subprocess.PIPE)
#print(p.stdout.read())
output=p.stdout.read()
#os.system(cmd)
#command="parallel -j 8 hmmscan "+"--cpu 1"+" --tblout {}.tab " + "../amoCAB {} ::: "+fnames
#print(cmd)


#splitting output with reges, since hmmscan sometimes truncates spaces


chunks=re.split(r"#\s+--- full sequence ---- --- best 1 domain ---- --- domain number estimation ----", output.decode())

#sorting here might be useles
total=[]
df_scramb=pd.DataFrame()
test_cnt=0
for chunk in chunks:
    if chunk != '':
        #print((chunk))

        data=chunk.split('\n')[3:-11]
#         if test_cnt <2:
#             print(data)
        test_cnt+=1
        #print(data)
        if data:
            d=pd.DataFrame([x.split() for x in data])
        else:
            continue

        query=re.findall('# Query file:\s*(.*)', chunk)[0]
        label=files2[query]
        #print(query)
        #d['query file']=query
        #df.insert(loc=idx, column='A', value=new_col)
        d.insert(loc=0, column='query file', value=query)
        d.insert(loc=1, column='label', value=label)
        #d['label']=files2[query]
        df_scramb=df_scramb.append(d)
        #total+=chunk.split("\n")[3:-11]

#lots of things here are just for compatibility with no-split version, 
#they need to be replaced!!!
#check for an empty dataframe
if df_scramb.empty:
    #print("\nNo hits found in the Input File ["+input_file+"]")
    sys.exit()
df_scramb.sort_values(by='query file', inplace=True)#to fix performance warning
df_scramb.set_index(['query file', 'label'], inplace=True)

#df_scramb[18]

df_scramb.iloc[:,18]=df_scramb.iloc[:,18:].apply(joinDescriptionColumns, axis=1)#this part is also redundant
df_scramb=df_scramb.iloc[:,:19]

#This block of code is redundant since we introduced positional iformation as fasta ids
#df_scramb.sort_index(level=1, inplace=True)#sort by label and then reset index
#df_scramb.reset_index(inplace=True)


#processing positional fasta ids
df_scramb[2]=df_scramb[2].astype(int)
df_scramb.reset_index(inplace=True)
df_scramb.sort_values(by=2, inplace=True)


##hmmscan output df is combined with seq. features extracted from input_file
#entire SeqIO.seq object is placed into 'seq' column
names=df_scramb[2].tolist()
file_iter = SeqIO.parse(open(input_file),"fasta")
df_scramb=df_scramb.merge(extractFeatures4(names, file_iter), left_on=2, how='inner', right_on='position', suffixes=('',''))
file_iter.close()
df_scramb.sort_values('position', inplace=True)#should be already sorted, unnecessary
#df_scramb.head()



#df_scramb.sort_values('position')
model2type={'AmoC': 'C', 'AMO': 'A', 'Monooxygenase_B': 'B'}



l=df_scramb['position'].tolist()

#### SECTION
# this section is kept solely of the sake of rel_coordinates functionality, and may be removed
# operon counting ignores this section
df_scramb['diff1']=abs((np.array(l)-np.array([0] + l[:-1])))#shift to left
df_scramb['diff2']=abs(np.array(l)-np.array(l[1:]+[0]))#shift to right
#df[(df['diff1'] <= 2) | (df['diff2'] <= 2)]
df_scramb['is_operon']=df_scramb[['diff1', 'diff2']].apply(lambda x: x < max_distance).apply(is_operon, axis=1)

df_scramb['rel_coordinates']=df_scramb[['diff1', 'diff2']].apply(rel_coordinates, args=(max_distance, ), axis=1)# passing a parameter into apply

#### END of Section


#df_scramb['operon_count']=operonCount(df_scramb['is_operon'].tolist(), df_scramb['position'].tolist())
#df_scramb['operon_count']=operonCount2(df_scramb['is_operon'].tolist(), df_scramb['position'].tolist(), max_distance, min_operon_size)

# operon identification and counting takes place here, positional sorted hit ids are taken as an input
df_scramb["operon_count"]=operonCount5(df_scramb[2])

df_scramb['type']=df_scramb[0].apply(lambda x: model2type[x])
df_scramb.head()


#groupping by combined operon df into operons by count and placing them into a list for processing
operons=list()
#output=pd.DataFrame()
for count,frame in df_scramb.groupby('operon_count', sort=False):
    if count > 0:

        operons.append(frame.copy())

#ops=list(operons)

#check if dot is in the filename
#if output_name is specified
if output_name:
    bn_lst=[output_name]
#standard output file names if not specified
else:
    bn_lst=basename.split(".")
    bn_lst[0]="cab_"+bn_lst[0]
#splitting basename into a lists

if len(bn_lst)>1:
    outfile1='.'.join(bn_lst[:-1])+".tsv"
    outfile2='.'.join(bn_lst[:-1])+".fasta"
    outfile3='.'.join(bn_lst[:-1])+".tab"
else:
    outfile1='.'.join(bn_lst)+".tsv"
    outfile2='.'.join(bn_lst)+".fasta"
    outfile3='.'.join(bn_lst)+".tab"

#print(frame)




/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:69: DeprecationWarning: 'U' mode is deprecated


# Body of the main function: Part 2

In [162]:
if operons != []:
    final_frame, output_list=operonParser(operons, basename)
else:
    final_frame=None
    output_list=None
    print("\nNo operons detected in the Input File ["+input_file+"]")
    #need to convert this block into a function
    if 'all' not in operon_type:
        df_scramb['seq']=df_scramb['seq'].apply(lambda x: str(x.seq)) #converting SeqIO object to string
        df_scramb['query file']=input_file
        try:
            df_scramb.to_csv(outfile3[4:], sep='\t') # remove cab_ from filename

        except:
            print("ERROR: could not create output file ["+outfile3[4:]+"]")
        sys.exit()







#    
#    if len(bn_lst)>1:
#        outfile1='.'.join(bn_lst[:-1])+".tsv"
#        outfile2='.'.join(bn_lst[:-1])+".fasta"
#        outfile3='.'.join(bn_lst[:-1])+".tab"
#    else:
#        outfile1='.'.join(bn_lst)+".tsv"
#        outfile2='.'.join(bn_lst)+".fasta"
#        outfile3='.'.join(bn_lst)+".tab"

#outfile1 = ".tsv"
#outfile2 = "out4.fasta"

#if we got till here we should have some data in operons list
#it might be not abc/cab though
if ((isinstance(final_frame, pd.DataFrame)) and (not final_frame.empty)):
    try:
        final_frame.to_csv(outfile1, sep='\t', header=True)
    except:
        print("ERROR: could not create output file ["+outfile1+"]")
elif operons !=[]: #create empty file to indicate some operons but not the ones of /cab,abc/
    with open(outfile1, 'w'):
        os.utime(outfile1, None)

#if len(ids)==len(sequences):
records=list()

if ((isinstance(output_list,list)) and (output_list !=[])):    
    try:
        with open(outfile2, "w") as output_handle:
            SeqIO.write(output_list, output_handle, "fasta")
    except:
        print("ERROR: could not create output file ["+outfile2+"]")

elif operons!=[]: #referring back to the groubpy statement: if there are operons, create empty file
    with open(outfile2, 'w'):
        os.utime(outfile2, None)        

if 'all' in operon_type:
    df_scramb['seq']=df_scramb['seq'].apply(lambda x: str(x.seq)) #converting SeqIO object to string
    df_scramb['query file']=input_file
    try:
        df_scramb.to_csv(outfile3, sep='\t')
    except:
        print("ERROR: could not create output file ["+outfile3+"]")



####closing temporary files#####            
for (l,f) in files:
    #print(l)
    #print(f.name)
    f.close()

In [167]:
operons=[]
operons

[]

In [168]:

for count,frame in df_scramb.groupby('operon_count', sort=False):
    if count > 0:

        operons.append(frame.copy())

In [169]:
operons[0]

,query file,label,0,1,2,3,4,5,6,7,...,descr,position,qid,seq,diff1,diff2,is_operon,rel_coordinates,operon_count,type
3,/tmp/tmpncpib233,0,Monooxygenase_B,PF04744.11,9824,-,9.9e-125,403.9,0.1,1.1e-124,...,draft_100040163,9824,draft_100040163,"(V, G, A, K, K, Q, T, V, M, A, A, K, K, I, R, ...",2189,1,True,1,1.0,B
4,/tmp/tmpncpib233,0,AMO,PF02461.15,9825,-,6.1e-82,262.6,17.4,8e-82,...,draft_100040164,9825,draft_100040164,"(M, N, E, V, Q, S, E, E, L, R, A, K, E, V, K, ...",1,1,True,1,1.0,A
5,/tmp/tmpncpib233,0,AmoC,PF04896.11,9826,-,1.5e-69,221.7,18.6,1.8e-69,...,draft_100040165,9826,draft_100040165,"(M, A, Q, T, H, A, T, N, D, L, A, V, G, S, G, ...",1,2190,True,1,1.0,C


In [170]:
def operonParser(op_lst, input_file, op_type=False, full_operons=False, remove_terminal_stop_codon=True):
    
    
    output_lst=list()#list of seq elements for fasta exporting
    out=pd.DataFrame()
    
    #initializing output variables
    if op_lst == []:
        return 1
    if full_operons:
        out= pd.DataFrame(columns=['operon #','input_file','type',  'rel_distance','size', 'concat_ids','concat_eval', 'concat_descr', 'C', 'A', 'B', 'full_operon'])
    out= pd.DataFrame(columns=['operon #','input_file','type','rel_distance','concat_eval','concat_ids','size',  'concat_descr', 'C', 'A', 'B'])
    
    for operon in op_lst:
        
        #make a copy so that no type error occurs
        operon=operon.copy()
        #print(operon)
        operon_structure=''.join(operon['type'].tolist())
        #print(operon_structure)
        op_type,forward=analyzeStructure(operon_structure)
        if op_type:
            #converting evalue to float
            operon[4]=operon[4].astype(float)#for correct sorting
            #reverse operon if needed
            #print(operon)
            if forward == -1:
                #''.join(operons[1].reset_index(drop=True)['type'].sort_index(ascending=False).tolist())
                operon=operon.reset_index(drop=True).sort_index(ascending=False)#rookie way   
            else:
                operon=operon.reset_index(drop=True) #else clause is a bit unnecessary
            #print(type(operon))
            cnt=int(operon['operon_count'].mean())#!!! needs an assert for integer value
            inp_file=input_file
            
            #ot=operon_structure #inclues extended understanding of an operon
            ot=op_type
            
            
            rd=''.join(operon.loc[:, 'rel_coordinates'].astype(str).tolist())
            sz=len(operon.index)
            
            
            idx_a=operon[operon.loc[:,'type']=='A'].sort_values(4, ascending=True).head(1).index[0]
            idx_b=operon[operon.loc[:,'type']=='B'].sort_values(4, ascending=True).head(1).index[0]
            idx_c=operon[operon.loc[:,'type']=='C'].sort_values(4, ascending=True).head(1).index[0]
            
            
            
            concat_eval= ";".join([str(x) for x in operon.loc[[idx_c, idx_a, idx_b]][4].astype(str)])
            #peron
            
            
            concat_ids=";".join([x.id for x in operon.loc[[idx_c, idx_a, idx_b]]['seq']])
            concat_descr=";".join([x.description for x in operon.loc[[idx_c, idx_a, idx_b]]['seq']])
            #str(operons[1][operons[1]['type']=='C']['seq'].values[0].seq)
            #takes one value from operon with the smallest e-vaue if multiple values are present
            #str(record.seq)
#             c_seq=str(operon[operon.loc[:,'type']=='C'].sort_values(4, ascending=True).head(1)['seq'].values[0].seq)
#             a_seq=str(operon[operon.loc[:,'type']=='A'].sort_values(4, ascending=True).head(1)['seq'].values[0].seq)
#             b_seq=str(operon[operon.loc[:,'type']=='B'].sort_values(4, ascending=True).head(1)['seq'].values[0].seq)


            #removing terminal stop codon symbol for concatenation
            c_seq=str(operon.loc[idx_c]['seq'].seq)
            a_seq=str(operon.loc[idx_a]['seq'].seq)
            b_seq=str(operon.loc[idx_b]['seq'].seq)
            
            if remove_terminal_stop_codon:
                if c_seq[-1] == "*":
                    c_seq=c_seq[:-1]
                if a_seq[-1] == "*":
                    a_seq=a_seq[:-1]
                if b_seq[-1] == "*":
                    b_seq=b_seq[:-1]
        
            record=SeqRecord(Seq(c_seq+a_seq+b_seq, IUPAC.protein), id=concat_ids, description=concat_descr)
#           records.append(record)
            
            output_lst.append(record)
#             if full_operons:
#                 full_op=
            
            
            out=out.append(pd.Series({'operon #':cnt, 
                                  'input_file':inp_file, 
                                  'type':ot, 
                                  'rel_distance':rd, 
                                  'size': sz,
                                  'concat_eval': concat_eval,
                                  'concat_ids': concat_ids,
                                  'concat_descr': concat_descr,
                                  'C':c_seq,
                                  'A':a_seq,
                                  'B':b_seq}), ignore_index=True)
                                  
          
            
    return out, output_lst  

In [128]:
t='cabcabc'
operon_types={'abc','cba','cab', 'bac'}
for op in operon_types:
    print(t.find(op, 0))

-1
0
-1
1


In [129]:
f, o=operonParser(operons, basename)

NameError: name 'operonParser' is not defined

In [130]:
def find_subOperon(inp, types, start):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-130-4a7aca440f45>, line 2)

In [131]:
output=[]
for op in operon_types:
    pos=t.find(op)
    if pos != -1:
        output.append([(op, pos)])
output

[[('cab', 0)], [('abc', 1)]]

In [132]:
t='cabcab'

In [133]:
[[(x, t.find(x))] for x in operon_types if t.find(x) != -1]

[[('cab', 0)], [('abc', 1)]]

In [136]:
type(operon_types)

set

In [134]:
import numpy as np
output=[[(x, t.find(x))] for x in operon_types if t.find(x) != -1] #initialize list
for i in range(len(output)): # looping for the length of the possible operon list
        #print(output[i])
    min_pos=np.inf
    for op in operon_types:
        
        pos=t.find(op, (output[i][-1][1]+len(output[i][-1][0])))
        #min_pos=pos
        if (pos !=-1) and (pos < min_pos):
            min_pos=pos
            min_op=op
    if (min_pos !=-1) and (min_pos != np.inf) :
        output[i]+=[(min_op, min_pos)]

In [135]:
output

[[('cab', 0), ('cab', 3)], [('abc', 1)]]

In [62]:
maxlen=0
for i in range(len(output)):
    ln=len(output[i])
    if  ln > maxlen:
        maxlen=ln
        max_ind=i
        

In [141]:
min(list(map(len, operon_types)))

3

In [195]:
def multiple_operons_lookup(raw_operon_code, operon_types,overlapping=False):
    '''
    function takes the code names of the composite operon and the accepted operon types
    and returns a list with the possible combinations of simple operons found within composite operon.
    No overlapping genes is allowed by default.
    '''
    #how many full operons we can fit into raw_operon_code variable
    #depending on how we are looking: overlapping vs non-overlapping
    if overlapping: 
        lookup_depth=int(len(raw_operon_code)-min(list(map(len, operon_types)))+1)
    else:
        lookup_depth=int(len(raw_operon_code)/min(list(map(len, operon_types)))) #calculating maximum lookup depth
    print(lookup_depth)
    output=[[(x, t.find(x))] for x in operon_types if t.find(x) != -1] #initialize list
    lookup_depth -=1
    while lookup_depth > 0: # enter the loop if multiple operons could be found
        for i in range(len(output)): # looping for the length of the possible operon list
                #print(output[i])
            min_pos=np.inf
            for op in operon_types: # looping for the possible operon types
                if overlapping:
                    pos=t.find(op, (output[i][-1][1]+1))
                else:
                    pos=t.find(op, (output[i][-1][1]+len(output[i][-1][0])))
                #min_pos=pos
                if (pos !=-1) and (pos < min_pos):
                    min_pos=pos
                    min_op=op
            if (min_pos !=-1) and (min_pos != np.inf) :
                output[i]+=[(min_op, min_pos)]
        lookup_depth -=1
    return output

In [204]:
t='cabcab'
operon_types={'abc','cba','cab', 'bac'}

outp=multiple_operons_lookup(t, operon_types, overlapping=False)
outp

2


[[('cab', 0), ('cab', 3)], [('abc', 1)]]

In [203]:
operon_types

{'abc', 'bac', 'cab', 'cba'}

In [118]:
def find_all(a_str, sub_lst):
    start = 0
    while True:
        for sub in sub_lst:
            start = a_str.find(sub, start)
            if start == -1: 
                return
            yield (start, sub)
            start += len(sub) # use start += 1 to find overlapping matches

list(find_all('tramspam spam spam spam', ['sp', 'pa'])) # [0, 5, 10, 15]

[(4, 'sp'), (10, 'pa'), (14, 'sp'), (20, 'pa')]

In [25]:
t

'cabcab'

In [26]:
d={}

In [ ]:
pos= t.find('abc',0):
if pos != -1:
    d['abc']=pos
    pos = t.find('abc')

In [15]:
list(find_all('spamspamspamspam', 'spam')) 

[(0, 'spam'), (4, 'spam'), (8, 'spam'), (12, 'spam')]

In [173]:
operons[0]

,query file,label,0,1,2,3,4,5,6,7,...,descr,position,qid,seq,diff1,diff2,is_operon,rel_coordinates,operon_count,type
3,/tmp/tmpncpib233,0,Monooxygenase_B,PF04744.11,9824,-,9.9e-125,403.9,0.1,1.1e-124,...,draft_100040163,9824,draft_100040163,"(V, G, A, K, K, Q, T, V, M, A, A, K, K, I, R, ...",2189,1,True,1,1.0,B
4,/tmp/tmpncpib233,0,AMO,PF02461.15,9825,-,6.1e-82,262.6,17.4,8e-82,...,draft_100040164,9825,draft_100040164,"(M, N, E, V, Q, S, E, E, L, R, A, K, E, V, K, ...",1,1,True,1,1.0,A
5,/tmp/tmpncpib233,0,AmoC,PF04896.11,9826,-,1.5e-69,221.7,18.6,1.8e-69,...,draft_100040165,9826,draft_100040165,"(M, A, Q, T, H, A, T, N, D, L, A, V, G, S, G, ...",1,2190,True,1,1.0,C


In [175]:
f

,operon #,input_file,type,rel_distance,concat_eval,concat_ids,size,concat_descr,C,A,B
0,1,43209.assembled.faa,CAB,111,1.5e-69;6.1e-82;9.9e-125,draft_100040165;draft_100040164;draft_100040163,3,draft_100040165;draft_100040164;draft_100040163,MAQTHATNDLAVGSGESFGSKTAGHFRRNRIAWGFGALVLAVGAGL...,MNEVQSEELRAKEVKEARAQILDMESLNNPAAMKLYRRLDGILILV...,VGAKKQTVMAAKKIRSRLGLTSIVLSALVLFSGEVLAHGERAQLAS...
1,3,43209.assembled.faa,CAB,111,1.9e-91;2.1e-86;2.3e-133,draft_100068514;draft_100068513;draft_100068512,3,draft_100068514;draft_100068513;draft_100068512,MSAVMKSTERGGAGLQSARKIFNAWPVALGVAGVAALLLFYRLYQG...,MTANDQAVLDVAEMSPKIAKWSRTMDILVVIVAAFLIMSVSHIGFL...,MILKFRKIILACLAVAALLISVFLPQTAAAHGERATEPSIRTRSVH...
2,4,43209.assembled.faa,CAB,111,1.1e-82;1e-82;6.4e-126,draft_100084335;draft_100084336;draft_100084337,3,draft_100084335;draft_100084336;draft_100084337,MKTSSIKDVGAVAAPGVAWGKFFAVLIGLAVIFTVYRVYSEATAFT...,MSADVSSRIDAQVDWVVIPLVVVLLGSIFSFEFALLVGDWDYWIDW...,MKFFTKLFALVQGLLLASTMLCALPAQAHGERAQQPALRMRTVHWF...


0          951
1          955
2         7635
3         9824
4         9825
5         9826
6        12016
7        18133
8        18134
9        18340
10       18341
11       18342
12       21829
13       21830
14       21831
15       29736
16       37838
17       40388
18       70966
19       72480
20      105576
21      107807
22      132227
23      141562
24      169812
25      192936
26      212081
27      215069
28      222084
29      235570
        ...   
173    1568979
174    1579312
175    1594480
176    1594481
177    1597232
178    1609945
179    1614706
180    1620962
181    1621038
182    1630420
183    1631839
184    1636700
185    1636701
186    1648194
187    1648195
188    1661751
189    1661752
190    1662629
191    1676853
192    1704676
193    1707516
194    1714062
195    1721368
196    1739494
197    1763108
198    1775384
199    1776543
200    1797504
201    1804802
202    1805932
Name: 2, Length: 203, dtype: int64

In [124]:
# all in one function, identifies and counts
def operonCount5(inp, position=1):#recursive operon count
    '''
    takes a series, np.array or list of sorted positions as an input and returns 
    a numpy array of integers, where 0 indicates  no operon, and a positive value indicates operon number
    
    '''
    beginning=True
    inp_size=len(inp)
    out=np.empty([inp_size])
    out[0]=0 # taking care of the initial value
    op_cnt=0
    operon=0
    while position < (inp_size): # i < 203## we have 1 - 202, and checking for previous position
        

        if (inp[position] - inp[position-1]) <= min_operon_size:
            
            if (operon==0): #if starting a new operon appending a previous position
                op_cnt+=1
                out[position-1]=op_cnt #capturing initial position, and re-writing previous False
            operon=op_cnt
        else:
            operon=0
        out[position]=operon
        position+=1
    return out

In [125]:
operonCount5(df_scramb[2])
#operonCount5([124, 125, 226, 337, 7138, 99139])

array([  0.,   0.,   0.,   1.,   1.,   1.,   0.,   2.,   2.,   3.,   3.,
         3.,   4.,   4.,   4.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   5.,   5.,   0.,   0.,   0.,   0.,   0.,   6.,
         6.,   0.,   0.,   0.,   7.,   7.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   8.,
         8.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   9.,
         9.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,  10.,  10.,   0.,   0.,   0.,   0.,   0.,
         0.,  11.,  11.,   0.,  12.,  12.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   

In [102]:
np.zeros([5])

array([ 0.,  0.,  0.,  0.,  0.])

In [83]:
df_scramb[2][180:]

180    1620962
181    1621038
182    1630420
183    1631839
184    1636700
185    1636701
186    1648194
187    1648195
188    1661751
189    1661752
190    1662629
191    1676853
192    1704676
193    1707516
194    1714062
195    1721368
196    1739494
197    1763108
198    1775384
199    1776543
200    1797504
201    1804802
202    1805932
Name: 2, dtype: int64

In [53]:
def operonCount4(is_opr, inp):
    '''
    takes a boolean aray of values indicting operon elements and counts operons based on the positional information
    and
    the total hits ordered positions list
    '''
    #make a copy of an input array
    op_cnt=np.copy(is_opr)# basically for False values
    rel_positions=np.where(is_opr==True)[0]
    print(rel_positions)
    op_counter=0# start counting from 1, not 0

    for rel_pos in rel_positions:
        if rel_pos == 0: #capturing the beginning of the list clause
            op_counter +=1
            op_cnt[rel_pos]=op_counter
       
    
        if (inp[rel_pos] - inp[rel_pos -1]) <= min_operon_size:
            op_cnt[rel_pos]=op_counter
        else:
            op_counter+=1
            op_cnt[rel_pos]=op_counter
    return op_cnt

In [54]:
x=np.array([False, True, True, False, False, False, True])
x

array([False,  True,  True, False, False, False,  True], dtype=bool)

In [55]:
x

array([False,  True,  True, False, False, False,  True], dtype=bool)

In [56]:
y=[10, 16, 17, 25, 37, 45, 70]

In [57]:
operonCount4(x, y)

[1 2 6]


array([False,  True,  True, False, False, False,  True], dtype=bool)

In [12]:
operons=list()
for i in range(len(df_scramb[2].index)):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
